In [27]:
import pandas as pd

In [28]:
allergies = pd.read_csv('allergies.csv')
careplans = pd.read_csv('careplans.csv')
claims = pd.read_csv('claims.csv')
conditions = pd.read_csv('conditions.csv')
encounters = pd.read_csv('encounters.csv')
immunizations = pd.read_csv('immunizations.csv')
medications = pd.read_csv('medications.csv')
observations = pd.read_csv('observations.csv')
patients = pd.read_csv('patients.csv')

In [29]:
column_names = ['START','STOP']
allergies_clean = allergies.drop(column_names, axis=1)
careplans_clean = careplans.drop(column_names, axis=1)
conditions_clean = conditions.drop(column_names, axis=1)
medications_clean = medications.drop(column_names, axis=1)

column_names = ['DATE']
observations_clean = observations.drop(column_names, axis=1)
immunizations_clean = immunizations.drop(column_names, axis=1)
encounters_clean = encounters.drop(column_names, axis=1)

In [30]:
patients = patients.rename(columns={'patient': 'PATIENT'})

def add_suffix(df, suffix):
    renamed_columns = {}
    for col_name in df.columns:
        if col_name != 'PATIENT':
            renamed_columns[col_name] = col_name + '_' + suffix
        else:
            renamed_columns[col_name] = col_name
    return df.rename(columns=renamed_columns)


allergies_clean = add_suffix(allergies_clean, 'ALLERGIES')
careplans_clean = add_suffix(careplans_clean, 'CAREPLANS')
claims_clean = add_suffix(claims, 'CLAIMS')
conditions_clean = add_suffix(conditions_clean, 'CONDITIONS')
encounters_clean = add_suffix(encounters_clean, 'ENCOUNTERS')
immunizations_clean = add_suffix(immunizations_clean, 'IMMUNIZATIONS')
medications_clean = add_suffix(medications_clean, 'MEDICATIONS')
observations_clean = add_suffix(observations_clean, 'OBSERVATIONS')


In [43]:
merged_df = pd.merge(allergies_clean, patients, on='PATIENT', how='inner')
merged_df = pd.merge(merged_df, careplans_clean, on='PATIENT', how='inner')

merged_df

,PATIENT,ENCOUNTER_ALLERGIES,CODE_ALLERGIES,DESCRIPTION_ALLERGIES,birthdate,deathdate,ssn,drivers,passport,prefix,...,ethnicity,gender,birthplace,address,ID_CAREPLANS,ENCOUNTER_CAREPLANS,CODE_CAREPLANS,DESCRIPTION_CAREPLANS,REASONCODE_CAREPLANS,REASONDESCRIPTION_CAREPLANS
0,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,irish,M,Winchendon MA US,53328 Kristopher Springs Suite 264 Whitman MA ...,49d820d7-7aa7-46db-bfdc-125b316b30e8,9d87c22d-a777-426b-b020-cfa469229f82,711282006,Skin condition care,24079001.0,Atopic dermatitis
1,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,irish,M,Winchendon MA US,53328 Kristopher Springs Suite 264 Whitman MA ...,49d820d7-7aa7-46db-bfdc-125b316b30e8,9d87c22d-a777-426b-b020-cfa469229f82,225130001,Application of moisturizer to skin,24079001.0,Atopic dermatitis
2,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,irish,M,Winchendon MA US,53328 Kristopher Springs Suite 264 Whitman MA ...,ef225e49-eaee-45c6-99ee-2cf094851011,9d87c22d-a777-426b-b020-cfa469229f82,326051000000105,Self care,NaN,NaN
3,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,irish,M,Winchendon MA US,53328 Kristopher Springs Suite 264 Whitman MA ...,ef225e49-eaee-45c6-99ee-2cf094851011,9d87c22d-a777-426b-b020-cfa469229f82,58332002,Allergy education,NaN,NaN
4,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,irish,M,Winchendon MA US,53328 Kristopher Springs Suite 264 Whitman MA ...,2bbd4797-4353-4a5b-83ed-b64930f71222,9d87c22d-a777-426b-b020-cfa469229f82,326051000000105,Self care,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7005,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,english,F,Hardwick MA US,71970 Kub Trace Apt. 499 Pembroke MA 02359 US,1bfecb0b-4b29-4872-942d-b87e8b8060a4,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,710818004,Inhaled steroid therapy,233678006.0,Childhood asthma
7006,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,english,F,Hardwick MA US,71970 Kub Trace Apt. 499 Pembroke MA 02359 US,1bfecb0b-4b29-4872-942d-b87e8b8060a4,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,708409001,Home nebulizer therapy,233678006.0,Childhood asthma
7007,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,english,F,Hardwick MA US,71970 Kub Trace Apt. 499 Pembroke MA 02359 US,1bfecb0b-4b29-4872-942d-b87e8b8060a4,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,229298005,Breathing control,233678006.0,Childhood asthma
7008,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,english,F,Hardwick MA US,71970 Kub Trace Apt. 499 Pembroke MA 02359 US,7db39f6a-1425-4056-80c8-e32237c17121,96dcb7b2-cc41-436d-8e82-bec70a7a67a6,170836005,Allergic disorder monitoring,NaN,NaN


In [44]:
merged_df = pd.merge(merged_df, claims_clean, on='PATIENT', how='inner')
merged_df = pd.merge(merged_df, conditions_clean, on='PATIENT', how='inner')

In [33]:
#merged_df = pd.merge(merged_df, encounters_clean, on='PATIENT', how='inner')

In [45]:
merged_df

,PATIENT,ENCOUNTER_ALLERGIES,CODE_ALLERGIES,DESCRIPTION_ALLERGIES,birthdate,deathdate,ssn,drivers,passport,prefix,...,REASONDESCRIPTION_CAREPLANS,ID_CLAIMS,BILLABLEPERIOD_CLAIMS,ORGANIZATION_CLAIMS,ENCOUNTER_CLAIMS,DIAGNOSIS_CLAIMS,TOTAL_CLAIMS,ENCOUNTER_CONDITIONS,CODE_CONDITIONS,DESCRIPTION_CONDITIONS
0,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,Atopic dermatitis,bb46211f-0d65-447b-a615-c4a8fdc3ba2a,1991-11-03,temp organization,ab6d8296-d3c7-4fef-9215-40b156db67ac,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis
1,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,Atopic dermatitis,bb46211f-0d65-447b-a615-c4a8fdc3ba2a,1991-11-03,temp organization,ab6d8296-d3c7-4fef-9215-40b156db67ac,NaN,100,331c5000-2b26-42a9-bd64-84222ec0d550,195662009,Acute viral pharyngitis (disorder)
2,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,Atopic dermatitis,853218e1-0c5d-4825-b40a-3093caded16f,2007-11-10,temp organization,ab6d8296-d3c7-4fef-9215-40b156db67ac,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis
3,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,Atopic dermatitis,853218e1-0c5d-4825-b40a-3093caded16f,2007-11-10,temp organization,ab6d8296-d3c7-4fef-9215-40b156db67ac,NaN,100,331c5000-2b26-42a9-bd64-84222ec0d550,195662009,Acute viral pharyngitis (disorder)
4,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,Atopic dermatitis,d406461b-4b4f-4e13-82bc-02d076cdc3ef,2007-12-03,temp organization,ab6d8296-d3c7-4fef-9215-40b156db67ac,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318258,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,NaN,0ec20146-50c8-4010-8e9b-b30e06d92271,2017-07-27,temp organization,7d930372-7a58-4940-ac33-cd6373720f4a,NaN,100,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,233678006,Childhood asthma
1318259,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,NaN,0ec20146-50c8-4010-8e9b-b30e06d92271,2017-07-27,temp organization,7d930372-7a58-4940-ac33-cd6373720f4a,NaN,100,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,232353008,Perennial allergic rhinitis with seasonal vari...
1318260,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,NaN,0ec20146-50c8-4010-8e9b-b30e06d92271,2017-07-27,temp organization,7d930372-7a58-4940-ac33-cd6373720f4a,NaN,100,b5420df0-7f41-4dcf-bf7d-48fe80eee760,195662009,Acute viral pharyngitis (disorder)
1318261,7d930372-7a58-4940-ac33-cd6373720f4a,ca9ae5f5-8f5c-4d74-8d1b-9f820d9c60bc,424213003,Allergy to bee venom,1996-07-26,NaN,999-84-4314,S99981736,FALSE,Ms.,...,NaN,0ec20146-50c8-4010-8e9b-b30e06d92271,2017-07-27,temp organization,7d930372-7a58-4940-ac33-cd6373720f4a,NaN,100,7d36f2b0-5cab-4c21-ad04-e22d040b4840,195662009,Acute viral pharyngitis (disorder)


In [34]:
total_rows = len(encounters_clean)
rows_per_section = total_rows // 4
section1 = encounters_clean.iloc[:rows_per_section]
section2 = encounters_clean.iloc[rows_per_section:2*rows_per_section]
section3 = encounters_clean.iloc[2*rows_per_section:3*rows_per_section]
section4 = encounters_clean.iloc[3*rows_per_section:]

section1 = add_suffix(section1, '1')
section2 = add_suffix(section2, '2')
section3 = add_suffix(section3, '3')
section4 = add_suffix(section1, '4')


In [46]:
merged_df = pd.merge(merged_df, section1, on='PATIENT', how='inner')

In [47]:
merged_df

,PATIENT,ENCOUNTER_ALLERGIES,CODE_ALLERGIES,DESCRIPTION_ALLERGIES,birthdate,deathdate,ssn,drivers,passport,prefix,...,DIAGNOSIS_CLAIMS,TOTAL_CLAIMS,ENCOUNTER_CONDITIONS,CODE_CONDITIONS,DESCRIPTION_CONDITIONS,ID_ENCOUNTERS_1,CODE_ENCOUNTERS_1,DESCRIPTION_ENCOUNTERS_1,REASONCODE_ENCOUNTERS_1,REASONDESCRIPTION_ENCOUNTERS_1
0,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis,9d87c22d-a777-426b-b020-cfa469229f82,371883000,Outpatient procedure,NaN,NaN
1,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis,233fc27f-5067-4ab0-9b78-ab3aefbce2e6,371883000,Outpatient procedure,NaN,NaN
2,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis,0d20357f-be9c-4287-8840-d55bbd21748a,371883000,Outpatient procedure,NaN,NaN
3,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis,9a2fcc09-e958-43d9-a993-78e92073306d,371883000,Outpatient procedure,NaN,NaN
4,ab6d8296-d3c7-4fef-9215-40b156db67ac,9d87c22d-a777-426b-b020-cfa469229f82,425525006,Allergy to dairy product,1994-04-19,NaN,999-55-1956,S99951024,X10571629X,Mr.,...,NaN,100,9d87c22d-a777-426b-b020-cfa469229f82,24079001,Atopic dermatitis,80a1ec92-91a0-404d-b52e-4cedb08cfe7f,371883000,Outpatient procedure,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15894056,304ef151-bcfb-4f05-94f3-a7598efac77e,961482c8-dab8-4355-9358-91737f288349,419474003,Allergy to mould,1961-01-13,2010-07-05,999-66-9307,S99934260,X19125077X,Mr.,...,NaN,100,9a5aff91-2987-4fda-9f87-e2c9a45c04d2,423121009,Non-small cell carcinoma of lung TNM stage 4 ...,ec4d577c-805d-4c01-be1d-a96ba1d40c5c,185347001,Encounter for problem,NaN,NaN
15894057,304ef151-bcfb-4f05-94f3-a7598efac77e,961482c8-dab8-4355-9358-91737f288349,419474003,Allergy to mould,1961-01-13,2010-07-05,999-66-9307,S99934260,X19125077X,Mr.,...,NaN,100,9a5aff91-2987-4fda-9f87-e2c9a45c04d2,423121009,Non-small cell carcinoma of lung TNM stage 4 ...,11047037-3b3f-4566-b8f9-64a14d890e2f,185347001,Encounter for problem,NaN,NaN
15894058,304ef151-bcfb-4f05-94f3-a7598efac77e,961482c8-dab8-4355-9358-91737f288349,419474003,Allergy to mould,1961-01-13,2010-07-05,999-66-9307,S99934260,X19125077X,Mr.,...,NaN,100,9a5aff91-2987-4fda-9f87-e2c9a45c04d2,423121009,Non-small cell carcinoma of lung TNM stage 4 ...,fe51732a-e5f5-43cf-a9a3-006e75dacb75,185347001,Encounter for problem,NaN,NaN
15894059,304ef151-bcfb-4f05-94f3-a7598efac77e,961482c8-dab8-4355-9358-91737f288349,419474003,Allergy to mould,1961-01-13,2010-07-05,999-66-9307,S99934260,X19125077X,Mr.,...,NaN,100,9a5aff91-2987-4fda-9f87-e2c9a45c04d2,423121009,Non-small cell carcinoma of lung TNM stage 4 ...,36353701-62e8-4caa-a99b-e7e1b078333b,185347001,Encounter for problem,NaN,NaN


In [51]:
section2

,ID_ENCOUNTERS_2,PATIENT,CODE_ENCOUNTERS_2,DESCRIPTION_ENCOUNTERS_2,REASONCODE_ENCOUNTERS_2,REASONDESCRIPTION_ENCOUNTERS_2
5131,e6005940-246c-4f21-8332-a57ecacaab9a,f0438525-1d1c-4e55-81c5-ab5b8d15e7e8,698314001,Consultation for treatment,NaN,NaN
5132,d104c46b-1a2d-44d5-999f-d221d7027b73,f0438525-1d1c-4e55-81c5-ab5b8d15e7e8,305408004,Admission to surgical department,NaN,NaN
5133,e5de2d54-f66b-471a-8cd3-a14fecb6d3e0,f0438525-1d1c-4e55-81c5-ab5b8d15e7e8,308335008,Patient encounter procedure,NaN,NaN
5134,1e127368-da7c-4f5c-b48c-34ec0c79178e,f0438525-1d1c-4e55-81c5-ab5b8d15e7e8,308335008,Patient encounter procedure,NaN,NaN
5135,40195587-d173-41d5-b33e-b98ea55b7c00,f0438525-1d1c-4e55-81c5-ab5b8d15e7e8,185349003,Outpatient Encounter,NaN,NaN
...,...,...,...,...,...,...
10257,cc7578ac-dc24-4a8c-9c12-e4ccbd7b01eb,6de9fda5-570e-433b-a95e-aa09bfbcbc22,170258001,Outpatient Encounter,NaN,NaN
10258,10d3bcba-08a5-4a42-afa3-b7b74c3e1f3d,6de9fda5-570e-433b-a95e-aa09bfbcbc22,170258001,Outpatient Encounter,NaN,NaN
10259,51eb9c0d-8d75-49a1-a81b-ab0a351268e2,6de9fda5-570e-433b-a95e-aa09bfbcbc22,170258001,Outpatient Encounter,NaN,NaN
10260,5fdc3e42-5d18-4cda-8853-95f4085b50a3,6de9fda5-570e-433b-a95e-aa09bfbcbc22,170258001,Outpatient Encounter,NaN,NaN


In [49]:
merged_df = pd.merge(merged_df, section2, on='PATIENT', how='inner')

In [50]:
merged_df

,PATIENT,ENCOUNTER_ALLERGIES,CODE_ALLERGIES,DESCRIPTION_ALLERGIES,birthdate,deathdate,ssn,drivers,passport,prefix,...,ID_ENCOUNTERS_1,CODE_ENCOUNTERS_1,DESCRIPTION_ENCOUNTERS_1,REASONCODE_ENCOUNTERS_1,REASONDESCRIPTION_ENCOUNTERS_1,ID_ENCOUNTERS_2,CODE_ENCOUNTERS_2,DESCRIPTION_ENCOUNTERS_2,REASONCODE_ENCOUNTERS_2,REASONDESCRIPTION_ENCOUNTERS_2


In [37]:
merged_df = pd.merge(merged_df, section3, on='PATIENT', how='inner')

In [38]:
merged_df = pd.merge(merged_df, section4, on='PATIENT', how='inner')

In [39]:
merged_df = pd.merge(merged_df, immunizations_clean, on='PATIENT', how='inner')

In [40]:
merged_df = pd.merge(merged_df, medications_clean, on='PATIENT', how='inner')
merged_df = pd.merge(merged_df, observations_clean, on='PATIENT', how='inner')